In [1]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from utils import geo
import pandas as pd
import regionmask
from dask.diagnostics import ProgressBar

In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': False})

In [3]:
rootdir = '/local/data/globcolour/from_cmems/OCEANCOLOUR_GLO_CHL_L4_REP_OBSERVATIONS_009_082/SouthernOcean_30Sto65S/daily/in_months/'
filename = '*.nc'
def preprocess(ds):
    return ds.where(ds['time.month']==ds['time.month'][0],drop=True).chunk({'time':1})
ds = xr.open_mfdataset(rootdir+filename,preprocess=preprocess)
ds = ds.sel(lat=slice(-40,-70))
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,periodic='X')
ds['area'] = ds['dxC']*ds['dyC']

In [4]:
# Apply a landmask
mask = regionmask.defined_regions.natural_earth.land_110.mask(ds['lon'], ds['lat'])
mask = mask.where(np.isfinite(mask),1)
mask = mask.where(mask==1,np.nan)
# mask.plot()
ds['CHL'] = ds['CHL'].fillna(0)*mask

In [26]:
variable='CHL'
# Extract spatial average for each year
years = np.arange(1998,2022)
days = np.arange(0,365)
# Ordering such that years run from July to June
ds_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = xr.cftime_range(start,end)
    x = (ds[variable].sel({'time':slice(start,end)})).weighted(ds['area'].fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        ds_year.loc[{'year':year}]=x.values
ds_year.to_netcdf('../../data/globcolour_cmems_daily_'+variable+'_wmean-latlon_byyear-JULtoJUN_new.nc')

1998
[########################################] | 100% Completed | 43.5s
1999
[########################################] | 100% Completed | 49.4s
2000
[########################################] | 100% Completed | 57.5s
2001
[########################################] | 100% Completed |  1min  3.0s
2002
[########################################] | 100% Completed |  1min  1.5s
2003
[########################################] | 100% Completed |  1min  1.8s
2004
[########################################] | 100% Completed |  1min  2.6s
2005
[########################################] | 100% Completed |  1min  4.3s
2006
[########################################] | 100% Completed |  1min  2.0s
2007
[########################################] | 100% Completed |  1min  0.9s
2008
[########################################] | 100% Completed |  1min  6.9s
2009
[########################################] | 100% Completed |  1min  8.2s
2010
[########################################] | 100% Completed |  1m

### By basin

In [7]:
atlind = 20
indwpac = 170
wpacepac = -130
epacatl = -70

cond_atlantic = (ds['lon']>=epacatl) & (ds['lon']<atlind)
cond_indian = (ds['lon']>=atlind) & (ds['lon']<indwpac)
cond_wpacific = (ds['lon']>=indwpac) | (ds['lon']<wpacepac)
cond_epacific = (ds['lon']>=wpacepac) & (ds['lon']<epacatl)

atlantic = ds['CHL'].where(cond_atlantic,drop=True)
atlantic_area = ds['area'].where(cond_atlantic,drop=True)
indian = ds['CHL'].where(cond_indian,drop=True)
indian_area = ds['area'].where(cond_indian,drop=True)
wpacific = ds['CHL'].where(cond_wpacific,drop=True)
wpacific_area = ds['area'].where(cond_wpacific,drop=True)
epacific = ds['CHL'].where(cond_epacific,drop=True)
epacific_area = ds['area'].where(cond_epacific,drop=True)

In [11]:
# Extract spatial average for each year
years = np.arange(1998,2022)
days = np.arange(0,365)
# Ordering such that years run from July to June
atlantic_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
indian_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
wpacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
epacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name='CHL')
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = xr.cftime_range(start,end)
    
    # Atlantic
    print('atlantic')
    x = (atlantic.sel({'time':slice(start,end)})).weighted(atlantic_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        atlantic_year.loc[{'year':year}]=x.values
        
    # Indian
    print('indian')
    x = (indian.sel({'time':slice(start,end)})).weighted(indian_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        indian_year.loc[{'year':year}]=x.values
        
    # West Pacific
    print('wpacific')
    x = (wpacific.sel({'time':slice(start,end)})).weighted(wpacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        wpacific_year.loc[{'year':year}]=x.values
        
    # East Pacific
    print('epacific')
    x = (epacific.sel({'time':slice(start,end)})).weighted(epacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        epacific_year.loc[{'year':year}]=x.values

atlantic_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_atlantic.nc')
indian_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_indian.nc')
wpacific_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_wpacific.nc')
epacific_year.to_netcdf('../../data/globcolour_cmems_daily_CHL_wmean-latlon_byyear-JULtoJUN_epacific.nc')

1998
atlantic
[########################################] | 100% Completed | 34.5s
indian
[########################################] | 100% Completed | 31.7s
wpacific
[########################################] | 100% Completed | 26.5s
epacific
[########################################] | 100% Completed | 28.3s
1999
atlantic
[########################################] | 100% Completed | 37.3s
indian
[########################################] | 100% Completed | 37.4s
wpacific
[########################################] | 100% Completed | 33.3s
epacific
[########################################] | 100% Completed | 26.6s
2000
atlantic
[########################################] | 100% Completed | 36.9s
indian
[########################################] | 100% Completed | 42.2s
wpacific
[########################################] | 100% Completed | 36.9s
epacific
[########################################] | 100% Completed | 27.2s
2001
atlantic
[########################################] | 100% Com

In [9]:
x

<xarray.DataArray 'CHL' (time: 365)>
dask.array<dask_aware_interpnd, shape=(365,), dtype=float64, chunksize=(365,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1997-07-01 00:00:00 ... 1998-06-30 00:00:00

In [10]:
atlantic_year

<xarray.DataArray 'CHL' (day: 365, year: 24)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.11890841,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.12285271,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.12347057,        nan,        nan, ...,        nan,        nan,
               nan]])
Coordinates:
  * year     (year) int64 1998 1999 2000 2001 2002 ... 2017 2018 2019 2020 2021
  * day      (day) int64 0 1 2 3 4 5 6 7 8 ... 357 358 359 360 361 362 363 364